# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'matplotlib'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# URL for GET requests to retrieve city data
url = "http://api.openweathermap.org/data/2.5/weather?"

#query URL
query_url = f"{url}appid={weather_api_key}&units=imperial&q="

# Lists 
lon = []
temp = []
temp_max = []
humidity = []
wind_speed = []
lat = []
date = []
country = []
cloudiness = []

# Loop through and request
print("Data Retrieval Searching")
print("--------------------------")
count = 0
set = 1
for index, city in enumerate(cities):
    count = count + 1
   
    if count == 51:
        count = 1
        set = set + 1
        time.sleep(10)
    print(f"Processing Record {count} of Set {set} | {city}")
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        temp_max.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        date.append(response['dt'])
        country.append(response['sys']['country'])
        cloudiness.append(response['clouds']['all'])
    except KeyError:
        print("City not found. searching again...")
        lat.append(np.nan)
        lon.append(np.nan)
        temp.append(np.nan)
        temp_max.append(np.nan)
        humidity.append(np.nan)
        wind_speed.append(np.nan)
        date.append(np.nan)
        country.append(np.nan)
        cloudiness.append(np.nan)
print("----------------------------")
print("Data Retrieval Complete")
print("--------------------------")


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_df = pd.DataFrame({
    "City": cities,
    "Lat": lat,
    "Lng": lon,
    "Max Temp": temp_max,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country": country,
    "Date": date,
})

# Drop any cities that were skipped due to no response
city_df = city_df.dropna(how="any")

# Export.csv file.
city_df.to_csv("./output_data/cities.csv")
city_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
city_df["Humidity"].describe()

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
current_date = datetime.date(datetime.now())

# Create a scatter plot for latitude vs max temperature.
x_values = city_df["Lat"]
y_values = city_df["Max Temp"]

fig1, ax1 = plt.subplots(figsize=(10,8))
plt.scatter(x_values, y_values, edgecolor="green", linewidth=1, marker="x", alpha=0.8)
plt.title(f"City Latitude vs Max Temperature {current_date}")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()

# Save fig
plt.savefig("./output_data/latitude_vs_max_temp.png")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
x_values = city_df["Lat"]
y_values = city_df["Humidity"]

fig1, ax1 = plt.subplots(figsize=(10,8))
plt.scatter(x_values, y_values, edgecolor="black", linewidth=1, marker="d", alpha=0.8)
plt.xlabel("Latitude")
plt.ylabel("Humidity Percent")
plt.title(f"Latitude vs Humidity {current_date}")
plt.grid()

# Save
plt.savefig("./output_data/lat_vs_humidity.png")
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Create a scatter plot for latitude vs cloudiness.
x_values = city_df["Lat"]
y_values = city_df["Cloudiness"]

fig1, ax1 = plt.subplots(figsize=(10,8))

plt.scatter(x_values, y_values, edgecolor="black", linewidth=1, marker="d", alpha=0.8)
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.title(f"Latitude vs Cloudiness {current_date}")
plt.grid()

# Save
plt.savefig("./output_data/lat_vs_cloudiness.png")
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
x_values = city_df["Lat"]
y_values = city_df["Wind Speed"]

fig1, ax1 = plt.subplots(figsize=(10,8))

plt.scatter(x_values, y_values, edgecolor="black", linewidth=1, marker="d", alpha=0.8)

plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.title(f"Latitude vs Wind Speed {current_date}")
plt.grid()

# Save
plt.savefig("./output_data/lat_vs_windspeed.png")
plt.show()

## Linear Regression

In [ ]:
# Create a function to create Linear Regression plots for remaining activities
def plot_linear_regression(x_values, y_values, x_label, y_label, hemisphere, text_coordinates, ylim=None):
    
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

    # regression values
    regress_values = x_values * slope + intercept
    
    line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
    
    # plot    
    fig1, ax1 = plt.subplots(figsize=(12,8))
    plt.scatter(x_values, y_values, edgecolor="black", linewidth=1, marker="d", alpha=0.8)
    plt.plot(x_values,regress_values,"r-")
    current_date = datetime.date(datetime.now())
    plt.title(f"{hemisphere} Hemisphere - {x_label} vs {y_label} {current_date}",fontsize = 15)
    plt.xlabel(x_label,fontsize=14)
    plt.ylabel(y_label,fontsize=14)
    if ylim is not None:
        plt.ylim(0, ylim)
    plt.annotate(line_eq, text_coordinates, fontsize=20, color="red")
    
    # Print r square value
    print(f"The r-squared is: {rvalue**2}")
    corr = st.pearsonr(x_values,y_values)
    print(f"The correlation between both factors is {round(corr[0],2)}")

        
#Northern and Southern DFs
northern_df = city_df.loc[city_df["Lat"] >= 0]
southern_df = city_df.loc[city_df["Lat"] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = northern_df["Lat"]
y_values = northern_df["Max Temp"]
plot_linear_regression(x_values, y_values, "Latitude", "Max Temp (F)", "Northern", (10, 10))

# Savefig
plt.savefig("./output_data/north_maxtemp.png")
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = southern_df["Lat"]
y_values = southern_df["Max Temp"]
plot_linear_regression(x_values, y_values, "Latitude", "Max Temp (F)", "Southern", (-52, 75))

# Save the figure
plt.savefig("./output_data/southern_max_temp.png")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df['Lat']
y_values = northern_df['Humidity']
plot_linear_regression(x_values, y_values, "Latitude", "Humidity (%)", "Northern",(50,50))
plt.savefig("./output_data/northern_humidity.png")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df['Lat']
y_values = southern_df['Humidity']
plot_linear_regression(x_values, y_values, "Latitude", "Humidity (%)", "Southern",(50, 50), 100)
plt.savefig("./output_data/southern_humudity.png)
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df['Lat']
y_values = northern_df['Cloudiness']
plot_linear_regression(x_values, y_values, "Latitude", "Cloudiness (%)", "Northern", (20, 60))

plt.savefig("./output_data/northern_cloudiness.png", bbox_inches="tight")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df['Lat']
y_values = southern_df['Cloudiness']
plot_linear_regression(x_values, y_values, "Latitude", "Cloudiness(%)", "Southern",(-45, 60))
plt.savefig("./output_data/southern_cloudiness.png", bbox_inches="tight")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df['Lat']
y_values = northern_df['Wind Speed']
plot_linear_regression(x_values, y_values, "Latitude", "Wind Speed (mph)", "Northern",(20, 25))
plt.savefig("./output_data/northern_wind_speed.png", bbox_inches="tight")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df['Lat']
y_values = southern_df['Wind Speed']
plot_linear_regression(x_values, y_values, "Latitude", "Wind Speed (mph)", "Southern",(-40, 25), ylim=40)
plt.savefig("./output_data/southern_wind_speed.png", bbox_inches="tight")
plt.show()